In [48]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad
import random
random.seed(0)

In [49]:
# generate random sublist from current list
def generate_new_list(original_list):
    new_list = []
    for element in original_list:
        if random.choice([True, False]):
            new_list.append(element)
    return new_list

In [50]:
# return cur_arg's SHAP AAE
def compute_SHAP(bag, topic_a, cur_arg):
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    arg_name_all = []

    topic_arg = model.BAG.arguments[topic_a]
    for arg in model.BAG.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    print(cur_arg)

    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")


    # record initial base scores
    arg_initial_weight = []
    for arg in model.BAG.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))


    # print all the initial base scores
    print("All the base score and strength in QBAF:")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))


    sample_count = 1000 # iterate 1000 times
    difference = []
    for i in range(sample_count):

        new_list = generate_new_list(arg_name_no_topic_no_cur) # choose random elements in arg_name_no_topic_arg
        print(f"new_list:{new_list}")

        if new_list == []:
            new_tuple = ()
        else:
            new_tuple = tuple(new_list)
        j = new_tuple

        # remove arg(j) from the current combination i
        for k in j: # first remove the argument from the combination
            model.BAG.remove_argument(k)

        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

        # compute sigma(S U {i})
        print("Initial and final strength (1st):")
        for arg in model.BAG.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength
        print(f"strength_before_drop_cur:{strength_before_drop_cur}")

        # remove cur arg
        model.BAG.remove_argument(cur_arg)

        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

        # compute sigma(S)
        print("Initial and final strength (2nd):")
        for arg in model.BAG.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength
        print(f"strength_after_drop_cur:{strength_after_drop_cur}")

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)
        print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")


        # recover BAG for the next iteration
        model.BAG = grad.BAG("../../bags/td_qbaf.bag")
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)


    print("----------------------------")

    print(f"difference:{difference}")
    attribution = sum(difference)/sample_count
    print(attribution)

    # reset to original initial weight
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    return attribution

In [51]:
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG("../../bags/td_qbaf.bag")
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 9.499999999999982
Argument(name=s0, weight=0.5, strength=0.6983561366720544)
Argument(name=s1, weight=0.5, strength=0.6983561366720544)
Argument(name=s2, weight=0.5, strength=0.6983561366720544)
Argument(name=s3, weight=0.5, strength=0.6984075829197717)
Argument(name=s4, weight=0.5, strength=0.6983866309035758)
Argument(name=s5, weight=0.5, strength=0.6983866309035758)
Argument(name=s6, weight=0.5, strength=0.5000006666489577)
Argument(name=s7, weight=0.5, strength=0.7211368661875142)
Argument(name=s8, weight=0.5, strength=0.7211368661875142)
Argument(name=s9, weight=0.5, strength=0.7211368661875142)
Argument(name=s10, weight=0.5, strength=0.7211368661875142)
Argument(name=c0, weight=0.0, strength=0.8135102276868453)
Argument(name=c1, weight=0.0, strength=7.654312473156645e-06)
Argument(name=c2, weight=0.0, strength=0.8135536884131115)
Argument(name=c3, weight=0.0, strength=6.897036405265413e-06)
Argument(name=c4, weight=0.0, strength=5.320066899352037e-

0.0009670739847200497

In [ ]:
# compute attribution for one arg
topic_a = "c5"
all_args = model.BAG.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg] = compute_SHAP(model.BAG, topic_a, cur_arg)
print(f"attribution: \n{attribution}")

In [53]:
print(f"base score of the topic_arg: {model.BAG.arguments[topic_a].initial_weight}")
# sum of all attributions
total_sum = sum(attribution.values())
print(f"sum of all attributions: {total_sum}")
print("--------------------------------------------")
print(f"sum of all attributions + BS: {total_sum+model.BAG.arguments[topic_a].initial_weight}")
print(f"strength of the topic_arg: {model.BAG.arguments[topic_a].strength}")

base score of the topic_arg: 0.0
sum of all attributions: 1.0500316718397253
--------------------------------------------
sum of all attributions + BS: 1.0500316718397253
strength of the topic_arg: 0.8922323980695945
